# INFO 2950 Final Project Appendix
Alex Joos (aaj46), Jasmine Pearson (jcp348), Darby Bayne (dpb99) 

---

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb

import plotly.express as px
import plotly.graph_objects as go

import re
from scipy.stats import pearsonr

-------------------------------

<a name="datacleaning"></a>
<a name="education"></a>

## **<div align="center">&#9658; Data Cleaning: Education &#9664;</div>**

This data came originally as a Numbers file. When saved as a CSV, we then had to split the columns by ";". Other than that, the data was already cleaned and we had no other changes to make. 

In [2]:
education_df = pd.read_csv("schooling_years.csv", delimiter=";")

education_df.head()

,Entity,Code,Year,avg_years_of_schooling
0,Afghanistan,AFG,1870,0.01
1,Afghanistan,AFG,1875,0.01
2,Afghanistan,AFG,1880,0.01
3,Afghanistan,AFG,1885,0.01
4,Afghanistan,AFG,1890,0.01


We call head on the dataframe to visually confirm that the data was read in and deliminated correctly. 

In [3]:
education_df.to_csv("education_df", sep=',')

We save the dataframe as a csv file. 

-------------------------------

<a name="druguse"></a>

## **<div align="center">&#9658; Data Cleaning: Drug Use &#9664;</div>**

The data came in html format. Then, we read them using pd.read_html to access the files. This resulted in a list of dataframes. Since we only need to first data set, we sliced our input at the zeroth index and saved this to a new dataframe for each file. 

In [4]:
## loading in data for years 2017-2021
## html inputs as list of dfs, we only want the first one
drug_2021 = pd.read_html("drug_2021_stats.html")
stats_2021_df = drug_2021[0]

drug_2020 = pd.read_html("drug_2020_2019_stats.html")
stats_2020_df = drug_2020[0]

drug_2019 = pd.read_html("drug_2020_2019_stats.html")
stats_2019_df = drug_2019[0]

drug_2018 = pd.read_html("drug_2018_2017_stats.html")
stats_2018_df = drug_2018[0]

drug_2017 = pd.read_html("drug_2018_2017_stats.html")
stats_2017_df = drug_2017[0]

We then combined these dataframes on Drug using a left join. 

Then, we filtered the new data set to only include the Past Year and Drug columns. After, we renamed the Year columns to be more readable, and manually inputted the required years. 

Next, we melted the dataframe to make it long instead of wide and got rid of the drug column, as it was unnecesary since all data was based on the same drug. 

Then, we reordered the Years to account for changes made by manual adding. We also convert years to strings and numbers as number values to prevent errors in future graphing.

In [5]:
## getting data by joining on Drug column
combined_df = stats_2018_df.join(stats_2019_df.set_index('Drug'), 
                                 on='Drug', how='left', lsuffix='2017', 
                                 rsuffix='2018')

combined_df = combined_df.join(stats_2021_df.set_index('Drug'), 
                               on='Drug', how='left', rsuffix='2019')

combined_df = combined_df.filter(regex='Past Year|Drug', axis=1)

## getting data for only hallucinogenic drugs 
hal_df = combined_df[combined_df['Drug'] == "Hallucinogens"]
hal_df = hal_df.rename(columns = {'Past Year (2017)': '2017', 
                                   'Past Year (2018)': '2018',
                                   'Past Year (2019)': '2019',
                                   'Past Year (2020)': '2020',
                                   'Past Year (2021)': '2021'})

## manually inputing years 2013-2016 
hal_df['2016'] = 4903
hal_df['2015'] = 4692
hal_df['2014'] = 4250
hal_df['2013'] = 4430

## melting wide to long, renaming columns 
hal_df = pd.melt(hal_df, id_vars=['Drug'], var_name='Year', 
                 value_name='Number of Users (Thousands)')

## getting rid of drug category because its unecessary  
hal_df = hal_df[['Year', 'Number of Users (Thousands)']]

## since years go from 2021-2017 then 2016-2013, we sort by 
## years ascending, then reset the index 
hal_df = hal_df.sort_values(by='Year', ascending=True)
hal_df = hal_df.reset_index(drop=True)

## convert years to strings and numbers as number values to prevent errors
## when graphing 
hal_df['Year'] = hal_df['Year'].astype(str)
hal_df['Number of Users (Thousands)'] = pd.to_numeric(
                                    hal_df['Number of Users (Thousands)'], 
                                    errors='coerce')

## display 
hal_df.head()

,Year,Number of Users (Thousands)
0,2013,4430
1,2014,4250
2,2015,4692
3,2016,4903
4,2017,5125


We call head on the dataframe to visually confirm that the data was read in correctly and that all of our seperate dataframs and manually inputted information appear as desired. 



Later, we realized that we were unable to properly use this data as it was only for the United States and covered very limited years. So, it does not appear in our final analysis and so we do not bother making it into a csv file. 

-------------------------------

<a name="internet"></a>

## **<div align="center">&#9658; Data Cleaning: Internet Access &#9664;</div>**

This data was originally a csv file. First we read it in and display the first five rows to bettern understand all of the columns we will be working with. 

In [6]:
internet_df = pd.read_csv('internet_access_by_country 1989-2022.csv', 
                          thousands=",")
internet_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,83.780000,88.661227,93.542454,97.170000,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,12.237716,14.485849,15.753330,17.310987,20.063024,22.589591,24.988401,27.660654,NaN,NaN
2,Afghanistan,AFG,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,7.000000,8.260000,11.000000,13.500000,16.800000,17.600000,18.400000,NaN,NaN,NaN
3,Africa Western and Central,AFW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,14.923441,18.109485,22.869750,27.081219,32.089337,35.276448,41.749819,46.990497,NaN,NaN
4,Angola,AGO,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,21.400000,22.000000,23.200000,26.000000,29.000000,32.129392,32.550147,32.602302,NaN,NaN


The final Column, indicator name, and indicator code were uneeded for our research. We kept both of the country columns since we will likely use country code, but the names will be helpful for reference. 

Because we are joining the data on year and country, we wanted a year column, and to get that needed a vertical data frame.

We used pd.melt to put all of the years into the the "year" column, and their corresponding internet access percentage into the "internet access" column. We chose to keep NaN's because when joining the data frames together into a master it would be helpful to see which country-time mixes have full breadth of data. 

In [7]:
internet_df = internet_df.drop(['Unnamed: 67', 'Indicator Name', 'Indicator Code'], 
                               axis=1)

years_int = []
for year in range(1960, 2023):
    years_int.append(year)
years_str = [str(year) for year in years_int]

internet_df_melt = pd.melt(internet_df, 
                           id_vars=['Country Name','Country Code'], 
                           value_vars=years_str, 
                           var_name = 'year', 
                           value_name = 'internet access')
internet_df.rename(columns = {'year': 'Year'})
internet_df_melt.head()

,Country Name,Country Code,year,internet access
0,Aruba,ABW,1960,NaN
1,Africa Eastern and Southern,AFE,1960,NaN
2,Afghanistan,AFG,1960,NaN
3,Africa Western and Central,AFW,1960,NaN
4,Angola,AGO,1960,NaN


As done before, we continue to call head on the dataframe to visually make sure changes occurred as planned. 

In [8]:
internet_df.to_csv("internet_df", sep=',')

We save the dataframe as a csv file. 

-------------------------------

<a name="religion"></a>


## **<div align="center">&#9658; Data Cleaning: Religion &#9664;</div>**

First we read in the csv file. The data set originally had only rounded years up to 2010, so the sightings data will need a new column that shows the rounded year so religion can be matched to it.

In [9]:
religions_df = pd.read_csv('religions_by_country_1945-2010.csv', thousands=",")
religions_df.head()

,year,state,code,christianity_protestant,christianity_romancatholic,christianity_easternorthodox,christianity_anglican,christianity_other,christianity_all,judaism_orthodox,...,jainism_percent,confucianism_percent,syncretism_percent,animism_percent,noreligion_percent,otherreligion_percent,religion_sumpercent,total_percent,dual_religion,source_code
0,1945,United States of America,USA,66069671,38716742,1121898,2400000,1956807,110265118,821489,...,0.0,0.0,0.0,0.0,0.1635,0.0039,0.9961,1.0000,0,13
1,1950,United States of America,USA,73090083,42635882,3045420,3045420,1177214,122994019,1078078,...,0.0,0.0,0.0,0.0,0.1482,0.0041,0.9959,1.0000,0,18
2,1955,United States of America,USA,79294628,46402368,3454916,2572767,2277091,134001770,944000,...,0.0,0.0,0.0,0.0,0.1404,0.0193,0.9807,0.9999,0,15
3,1960,United States of America,USA,90692928,50587880,3334535,2710065,2908939,150234347,973500,...,0.0,0.0,0.0,0.0,0.1193,0.0076,0.9924,0.9999,0,13
4,1965,United States of America,USA,94165803,64761783,4792868,2822149,973155,167515758,991200,...,0.0,0.0,0.0,0.0,0.1020,0.0030,0.9970,1.0001,0,20


Then for each year-country pair, we looked through all of the non "all" religions. We chose not to do the all religions because they combine data from multiple others. Of all of the individual religions, the greatest population count per country was found and put into a new column called top_religion. We then changed the year column to rounded year so when this data joins to the other data frames it can map other years to the closest multiple of 5 year

In [10]:
religions = []

for i in range(len(religions_df.columns)):
    column_name = religions_df.columns[i]
    
    if column_name in ['year', 'state', 'code']:
        continue
    if (religions_df[column_name].dtype == float):
        religions_df[column_name]=pd.to_numeric(religions_df[column_name], 
                                downcast='integer')
    religions.append(religions_df[column_name])
    
    if column_name == 'religion_all':
        break
        
i = 0
eachrow = 0
topreligion = []
max_index = 0

#goes through all of the rows
for eachrow in range(religions_df.shape[0]): 
    valuelist = []
    
    #checks each religion
    for i in range(len(religions)): 
        #indexes the value of the religion in question
        value = religions_df.iloc[eachrow, 3+i] 
        valuelist.append(value)
        if ("all" in religions_df.columns[3+i]):
            valuelist[-1] = 0
    max_index = valuelist.index(max(valuelist))
    topreligion.append(religions_df.columns[3+ max_index])
    
top_df = religions_df[['year', 'code']].copy()
top_df['topReligion'] = topreligion
religion_df = top_df.rename(columns={'year': 'rounded year'})
religion_df.head()

,rounded year,code,topReligion
0,1945,USA,christianity_protestant
1,1950,USA,christianity_protestant
2,1955,USA,christianity_protestant
3,1960,USA,christianity_protestant
4,1965,USA,christianity_protestant


In [11]:
religion_df.to_csv("religion_df", sep=',')

We save the dataframe as a csv file. 

We call head on the dataframe to confirm all our changes have been made as desired.

Then, we printed the code column and all unique religions just to see what we were working with, and because we later planned to join on the code column of this dataframe.

In [12]:
print(religion_df['topReligion'].unique())
religion_df['code'].unique()

['christianity_protestant' 'christianity_romancatholic'
 'christianity_anglican' 'islam_other' 'islam_sunni'
 'christianity_easternorthodox' 'islam_shi’a' 'christianity_other'
 'judaism_orthodox' 'islam_ibadhi' 'buddhism_other' 'buddhism_mahayana'
 'buddhism_theravada']


array(['USA', 'CAN', 'BHM', 'CUB', 'HAI', 'DOM', 'JAM', 'TRI', 'BAR',
       'DMA', 'GRN', 'SLU', 'SVG', 'AAB', 'SKN', 'MEX', 'BLZ', 'GUA',
       'HON', 'SAL', 'NIC', 'COS', 'PAN', 'COL', 'VEN', 'GUY', 'SUR',
       'ECU', 'PER', 'BRA', 'BOL', 'PAR', 'CHL', 'ARG', 'URU', 'UKG',
       'IRE', 'NTH', 'BEL', 'LUX', 'FRN', 'MNC', 'LIE', 'SWZ', 'SPN',
       'AND', 'POR', 'GMY', 'GFR', 'GDR', 'POL', 'AUS', 'HUN', 'CZE',
       'CZR', 'SLO', 'ITA', 'SNM', 'MLT', 'ALB', 'MNG', 'MAC', 'CRO',
       'YUG', 'BOS', 'KOS', 'SLV', 'GRC', 'CYP', 'BUL', 'MLD', 'ROM',
       'RUS', 'EST', 'LAT', 'LIT', 'UKR', 'BLR', 'ARM', 'GRG', 'AZE',
       'FIN', 'SWD', 'NOR', 'DEN', 'ICE', 'CAP', 'STP', 'GNB', 'EQG',
       'GAM', 'MLI', 'SEN', 'BEN', 'MAA', 'NIR', 'CDI', 'GUI', 'BFO',
       'LBR', 'SIE', 'GHA', 'TOG', 'CAO', 'NIG', 'GAB', 'CEN', 'CHA',
       'CON', 'DRC', 'UGA', 'KEN', 'TAZ', 'BUI', 'RWA', 'SOM', 'DJI',
       'ETH', 'ERI', 'ANG', 'MZM', 'ZAM', 'ZIM', 'MAW', 'SAF', 'NAM',
       'LES', 'BOT',

-------------------------------

<a name="political"></a>


## **<div align="center">&#9658; Data Cleaning: Political Spectrum &#9664;</div>**

The original data set contained data of each selected country's democracy index, measuring Full democracies as 10-8.01, Flawed democracies as 8-6.01, Hybrid regimes from 6-4.01, Authoritarian regimes from 4-0, and No data as excluded countries. 

The code essentially runs through the steps from Discussion 3, by making sure its the right requests.get, then turns the file into text, then into html

In [13]:
wiki_democracy_url = "https://en.m.wikipedia.org/wiki/The_Economist_Democracy_Index"


wiki_democracy_result = requests.get(wiki_democracy_url)


if wiki_democracy_result.status_code != 200:
    print("something went wrong:", wiki_democracy_result)
    
wiki_democracy_text = wiki_democracy_result.text

wiki_democracy_html = BeautifulSoup(wiki_democracy_text, "html.parser")


country_democracy_list = wiki_democracy_html.find_all('table',
attrs={'class':"wikitable sortable",
        'style':"text-align:center;"})


country_democracy_df = pd.read_html(str(country_democracy_list))[1]

country_democracy_df.head()

,Region,2022 rank,Country,Regime type,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2008,2006
0,North America,12,Canada,Full democracy,8.88,8.87,9.24,9.22,9.15,9.15,9.15,9.08,9.08,9.08,9.08,9.08,9.08,9.07,9.07
1,North America,30,United States,Flawed democracy,7.85,7.85,7.92,7.96,7.96,7.98,7.98,8.05,8.11,8.11,8.11,8.11,8.18,8.22,8.22
2,Western Europe,20,Austria,Full democracy,8.20,8.07,8.16,8.29,8.29,8.42,8.41,8.54,8.54,8.48,8.62,8.49,8.49,8.49,8.69
3,Western Europe,36,Belgium,Flawed democracy,7.64,7.51,7.51,7.64,7.78,7.78,7.77,7.93,7.93,8.05,8.05,8.05,8.05,8.16,8.15
4,Western Europe,37,Cyprus,Flawed democracy,7.38,7.43,7.56,7.59,7.59,7.59,7.65,7.53,7.40,7.29,7.29,7.29,7.29,7.70,7.60


Now that we have confirmed that we have correctly scraped the data from wikipedia, we renamed columns to snake case to make them more readable and easier to refer to later on. 

In [14]:
country_democracy_df = country_democracy_df.rename(
        columns = {"Regime type":"Regime_Type",
        "2022 rank":"2022_Rank"})

Then, we took all of the year columns and turned them into one large year column in order to more easily merge the data together. This way, we can keep the democracy index values while also puting it aside the other values in the same year.

In [15]:
country_democracy_df = country_democracy_df.melt(
    id_vars = ["Region","2022_Rank","Country","Regime_Type"],
    value_vars = ["2022","2021","2020","2019","2018","2017",
                  "2016","2015","2014","2013","2012","2011","2010"], 
    var_name = "Year", 
    value_name = "Democracy_Index")

Finally, we drop columns that we don't need for our analysis: Region and 2022_Rank.

In [16]:
political_df = country_democracy_df.drop(
                ['Region', '2022_Rank'], 
                  axis=1)

political_df.head()

,Country,Regime_Type,Year,Democracy_Index
0,Canada,Full democracy,2022,8.88
1,United States,Flawed democracy,2022,7.85
2,Austria,Full democracy,2022,8.20
3,Belgium,Flawed democracy,2022,7.64
4,Cyprus,Flawed democracy,2022,7.38


We call head on the dataframe to confirm all our changes have been made as desired.

In [17]:
political_df.to_csv("scrubbed_democracy_df", sep=',')

We save the dataframe as a csv file. 

-------------------------------

<a name="wealth"></a>


## **<div align="center">&#9658; Data Cleaning: Wealth &#9664;</div>**

We follow the same loading process from a Wikipedia page as from our prior analysis of Political Spectrum. First, we handle data from years 2000 to 2009.

In [18]:
wiki_wealth_url = "https://en.wikipedia.org/wiki/List_of_countries_by_total_wealth"

wiki_wealth_result = requests.get(wiki_wealth_url)

if wiki_wealth_result.status_code != 200:
    print("something went wrong:", wiki_wealth_result)
    
wiki_wealth_text = wiki_wealth_result.text

wiki_wealth_html = BeautifulSoup(wiki_wealth_text, "html.parser")

country_wealth_list = wiki_wealth_html.find_all('table',
attrs={'class':"wikitable sortable"})

country_wealth_df_0to9 = pd.read_html(str(country_wealth_list))[0]

country_wealth_df_0to9.head()

,Year,Australia,Canada,China,France,Germany,India,Italy,Japan,South Korea,Spain,United Kingdom,United States,Aggregate share of the top 10
0,2000,—,2.2%,3.1%,4.0%,5.2%,—,4.7%,16.5%,1.5%,2.1%,5.6%,36.9%,81.8%
1,2001,—,2.1%,3.6%,4.0%,5.2%,—,4.6%,14.1%,1.5%,2.3%,5.6%,38.8%,81.8%
2,2002,—,2.0%,3.7%,4.6%,5.5%,—,5.3%,13.8%,1.7%,2.9%,6.1%,35.1%,80.7%
3,2003,1.9%,2.2%,3.7%,5.2%,5.8%,—,5.7%,12.9%,—,3.5%,6.2%,32.8%,79.9%
4,2004,1.9%,2.2%,3.9%,5.6%,5.7%,—,5.7%,11.5%,—,3.9%,6.5%,32.5%,79.4%


This next one is for years 2010-2019

In [19]:
country_wealth_df_10to19 = pd.read_html(str(country_wealth_list))[1]

country_wealth_df_10to19.head()

,Year,Australia,Canada,China,France,Germany,India,Italy,Japan,South Korea,Spain,United Kingdom,United States,Aggregate share of the top 10
0,2010,—,2.7%,10.1%,5.4%,4.7%,2.7%,4.6%,11.4%,—,3.5%,4.4%,25.7%,75.2%
1,2011,—,2.7%,11.9%,5.1%,4.6%,2.5%,4.3%,11.4%,—,3.4%,4.5%,25.2%,75.6%
2,2012,—,2.8%,12.5%,4.8%,4.6%,2.7%,4.2%,9.8%,—,2.9%,4.4%,25.6%,74.3%
3,2013,—,2.7%,13.8%,4.8%,4.7%,2.6%,4.1%,7.8%,—,2.7%,4.5%,26.8%,74.5%
4,2014,2.2%,2.6%,15.1%,4.2%,4.3%,2.8%,3.9%,7.1%,—,—,4.7%,28.6%,75.5%


Lastly, we handle years 2020-2022

In [20]:
country_wealth_df_20to22 = pd.read_html(str(country_wealth_list))[2]

country_wealth_df_20to22.head()

,Year,Australia,Canada,China,France,Germany,India,Italy,Japan,South Korea,Spain,United Kingdom,United States,Aggregate share of the top 10
0,2020,—,2.5%,17.5%,3.9%,4.3%,3.0%,2.9%,6.3%,2.3%,—,3.7%,29.9%,76.3%
1,2021,2.3%,2.7%,18.5%,3.5%,4.0%,3.2%,2.5%,5.4%,—,—,3.6%,31.3%,77.0%
2,2022,—,2.5%,18.6%,3.5%,3.8%,3.4%,2.4%,5.0%,2.2%,—,3.5%,30.8%,75.7%


Now we use the concat function to append all three to into one big data set covering years 2000 to 2022

In [21]:
combined1 = pd.concat([country_wealth_df_0to9, country_wealth_df_10to19], 
                      ignore_index=True)
combined = pd.concat([combined1,country_wealth_df_20to22],
                     ignore_index=True)

Lastly, we make a country column using the melt function in order to compare the values easier in later steps. We also strip away the % symbol and rename the column as percent to follow as we turn the values into floats.  

In [22]:
wealth_df = combined.melt(id_vars = ["Year"],
value_vars = ["Australia","Canada","China","France","Germany","India","Italy",
            "Japan","South Korea","Spain","United Kingdom","United States"],
var_name = "Countries", value_name = "Wealth_share_percent")

wealth_df["Wealth_share_percent"]=wealth_df["Wealth_share_percent"].str.rstrip("%")
wealth_df = wealth_df.replace("—", np.nan, regex=True)
wealth_df["Wealth_share_percent"] = wealth_df["Wealth_share_percent"].astype(float)
wealth_df.head()

,Year,Countries,Wealth_share_percent
0,2000,Australia,NaN
1,2001,Australia,NaN
2,2002,Australia,NaN
3,2003,Australia,1.9
4,2004,Australia,1.9


We call head on the dataframe to confirm all our changes have been made as desired.

In [23]:
wealth_df.to_csv("wealth_info", sep=',')

We save the dataframe as a csv file. 

-------------------------------

<a name="ufoone"></a>


## **<div align="center">&#9658; Data Cleaning: UFO Sightings, Part One &#9664;</div>**
This data was originally a csv file, and we kept it that way. From Kaggle, we chose to take the data that was already scrubbed, which accounted for missing cells with NaN values. We set low_memory=False as the duration column has different time formating (we do this for visual purposes to prevent an error message, since we do not use this column we do not need to fix the data to be in one unit). 

In [24]:
## load in scrubbed data 
scrubbed_location_df = pd.read_csv("scrubbed.csv", 
                                   low_memory=False) 
## make copy of df for later use 
time_df = scrubbed_location_df

scrubbed_location_df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


Then, we just pulled the columns pertinent to our analysis, datetime and country, from this data set. 

In [25]:
## getting just time and country data
scrubbed_location_df = duckdb.sql("SELECT datetime, country FROM\
                                scrubbed_location_df").df()

Now we add a new column with just the year in order to make it easier to graph the data and to merge it with other dataframes. This adjust_time function is from the original Kaggle link on this data set. The use of the function is original.

In [26]:
## adjusting times from datetime
def adjust_time(time_str):
    if "24:00" in time_str:
        time_str = time_str.replace("24:00", "00:00")
    return time_str

## adjust time
scrubbed_location_df['datetime'] = \
                    scrubbed_location_df['datetime'].apply(adjust_time)

## finish time formatting by subtracting year 
scrubbed_location_df['year'] = pd.to_datetime(scrubbed_location_df['datetime'], 
                                              format='%m/%d/%Y %H:%M').dt.year

scrubbed_location_df.head()

,datetime,country,year
0,10/10/1949 20:30,us,1949
1,10/10/1949 21:00,NaN,1949
2,10/10/1955 17:00,gb,1955
3,10/10/1956 21:00,us,1956
4,10/10/1960 20:00,us,1960


In [27]:
scrubbed_location_df.to_csv("scrubbed_location", sep=',')

We save the dataframe as a csv file. 

---

<a name="ufotwo"></a>

## **<div align="center">&#9658; Data Cleaning: UFO Sightings, Part Two &#9664;</div>**
Since the previous data only showed five countries, we set out to find a more inclusive dataset since the rest of our data (except for the drug use data, which we analyze seperately later) is international. We were unable to find such a dataset online, or even a table with more than just the data found in UFO Sightings Part One. So, instead, we noticed that the NUFORC had webpages for individual sightings. 

We read in country code map that allows us to produce a country code for each country name, later allowing us to merge on country code any data with only the country.

In [28]:
data = pd.read_csv('country_map.txt', delimiter='\t')
print(data.columns)
country_map = dict(zip(data['Country'], data['Code']))

print(country_map)

Index(['Country', '2', 'Code', 'num'], dtype='object')
{'Afghanistan': 'AFG', 'Albania': 'ALB', 'Algeria': 'DZA', 'American Samoa': 'ASM', 'Andorra': 'AND', 'Angola': 'AGO', 'Anguilla': 'AIA', 'Antarctica': 'ATA', 'Antigua and Barbuda': 'ATG', 'Argentina': 'ARG', 'Armenia': 'ARM', 'Aruba': 'ABW', 'Australia': 'AUS', 'Austria': 'AUT', 'Azerbaijan': 'AZE', 'Bahamas (the)': 'BHS', 'Bahrain': 'BHR', 'Bangladesh': 'BGD', 'Barbados': 'BRB', 'Belarus': 'BLR', 'Belgium': 'BEL', 'Belize': 'BLZ', 'Benin': 'BEN', 'Bermuda': 'BMU', 'Bhutan': 'BTN', 'Bolivia (Plurinational State of)': 'BOL', 'Bonaire, Sint Eustatius and Saba': 'BES', 'Bosnia and Herzegovina': 'BIH', 'Botswana': 'BWA', 'Bouvet Island': 'BVT', 'Brazil': 'BRA', 'British Indian Ocean Territory (the)': 'IOT', 'Brunei Darussalam': 'BRN', 'Bulgaria': 'BGR', 'Burkina Faso': 'BFA', 'Burundi': 'BDI', 'Cabo Verde': 'CPV', 'Cambodia': 'KHM', 'Cameroon': 'CMR', 'Canada': 'CAN', 'Cayman Islands (the)': 'CYM', 'Central African Republic (the)': 'C

We then wrote a loop that would go through each page, changing the id number at the end of the URL, and scrape the data using regex and beautiful soup so that we could get more diverse data on UFO sightings. We scraped just the location and the occurred time where location was valid (not just "local").

In [29]:
np.random.seed(2950)

df = pd.DataFrame(columns=["Occurred", "Location"])
base_url = "https://nuforc.org/sighting/?id="

session = requests.Session()  # Create a persistent session outside the loop

numbers = list(range(1, 1000001))
np.random.shuffle(numbers)

for number in range(89,1090):
    empty_series = pd.Series({
        "Occurred": "",
        "Location": ""
    })

    url = base_url + str(numbers[number])
    response = session.get(url)
   
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        occurredtext = str(soup.b.next_sibling.string)
        if occurredtext == 'Local':
            continue
        locationtext = str(soup.b.next_sibling.next_sibling.next_sibling.next_sibling.string)

        empty_series["Occurred"] = empty_series["Occurred"] + occurredtext
        empty_series["Location"] = empty_series["Location"] + locationtext
       
        df = pd.concat([df, empty_series.to_frame().T])

alien_df = df[df['Occurred'] != '  Local']
alien_df.head()

,Occurred,Location
0,2005-02-22 15:45:00 Local,"Netherlands, , Netherlands"
0,2020-05-30 23:15:00 Local,"Amelia (suburb of Cincinnati), OH, USA"
0,2020-03-05 20:17:00 Local,"Harrisburg, PA, USA"
0,2020-10-03 20:30:00 Local,"Ballston, NY, USA"
0,2017-05-10 05:35:00 Local,"Tampa, FL, USA"


We slice the data to get only the year and the country from our ufo sightings dataframe, as these are the values we plan on merging on later on. 

In [30]:
alien_df_copy = alien_df.copy()

def loc_fix(string, key):
    last_index = string.rfind(key)
    if last_index != -1:
        rest_of_string = string[last_index + len(key):]
        return rest_of_string.strip()
    else:
        return None

for number in range(0,(alien_df_copy.shape[0])):
    occur = alien_df_copy.iloc[number, 0]
    occur = occur[:occur.find("-")].strip()
    alien_df_copy.iloc[number, 0] = occur
    
    location = alien_df_copy.iloc[number, 1]
    location = loc_fix(location, ",")
    alien_df_copy.iloc[number, 1] = location
    
alien_df = alien_df_copy

We now update all locations in our alien_df to be in country codes instead of actual countries.

In [31]:
alien_df_copy = alien_df.copy()

alien_df_copy['Location'] = alien_df_copy['Location'].map(country_map).fillna(alien_df_copy['Location'])
alien_df = alien_df_copy

In order to get the sums of the numbers of sightings per year for each country, we first pivot the table to sum up each year as a categorical variable. Then, we melt that new table to create a tall dataframe with the sums of sightings for each year. 

In [32]:
pivot_df = alien_df.pivot_table(index="Location", columns="Occurred", aggfunc='size', fill_value=0)
pivot_df.columns = pivot_df.columns.astype(str)

melted_df = pivot_df.reset_index().melt(id_vars='Location', var_name='Year', value_name='Value')
melted_df = melted_df.rename(columns={'Location': 'Country Code', 'Value': 'Sightings'})
alien_df = melted_df
print(melted_df.head(10))
print(melted_df.iloc[:,2].unique())

      Country Code  Year  Sightings
0              ARG  1959          0
1              AUS  1959          0
2              BRA  1959          0
3              CAN  1959          0
4  Channel Islands  1959          0
5              DNK  1959          0
6              HRV  1959          0
7              IND  1959          0
8              NLD  1959          0
9              USA  1959          1
[ 0  1  2  3  4  5  9  7  6 11 12]


We save the dataframe as a csv file. 

In [33]:
alien_df.to_csv("alien_info", sep=',')

-----

<a name="combining"></a>

## **<div align="center">&#9658; Data Cleaning: Combining the Dataframes &#9664;</div>**

We make a list of our dataframes and print out the first rows to see what we're working with.

In [34]:
dfs = [education_df, internet_df, political_df, religion_df, wealth_df, alien_df]

print(education_df.head(1))
print("----------------------------------")
print(internet_df.head(1))
print("----------------------------------")
print(political_df.head(1))
print("----------------------------------")
print(religion_df.head(1))
print("----------------------------------")
print(wealth_df.head(1))
print("----------------------------------")
print(alien_df.head(1))

        Entity Code  Year  avg_years_of_schooling
0  Afghanistan  AFG  1870                    0.01
----------------------------------
  Country Name Country Code  1960  1961  1962  1963  1964  1965  1966  1967  \
0        Aruba          ABW   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   ...  2013   2014       2015       2016   2017  2018  2019  2020  2021  2022  
0  ...  78.9  83.78  88.661227  93.542454  97.17   NaN   NaN   NaN   NaN   NaN  

[1 rows x 65 columns]
----------------------------------
  Country     Regime_Type  Year  Democracy_Index
0  Canada  Full democracy  2022             8.88
----------------------------------
   rounded year code              topReligion
0          1945  USA  christianity_protestant
----------------------------------
   Year  Countries  Wealth_share_percent
0  2000  Australia                   NaN
----------------------------------
  Country Code  Year  Sightings
0          ARG  1959          0


We type cast all of the columns in order to make them the same type so that we can merge everything together. 

In [35]:
for df in dfs:
    df.columns = [np.str_(col) for col in df.columns]
for df in dfs:
    if 'Year' in df.columns:
        df['Year'] = df['Year'].astype(int)

We merge our education data. 

In [36]:
#adding educaiton
merged_df = pd.merge(alien_df, 
                     education_df, 
                     left_on=['Country Code', 'Year'], 
                     right_on=['Code', 'Year'], 
                     how='left')
merged_df.drop(columns='Code', inplace=True)

print(merged_df.head())

      Country Code  Year  Sightings Entity  avg_years_of_schooling
0              ARG  1959          0    NaN                     NaN
1              AUS  1959          0    NaN                     NaN
2              BRA  1959          0    NaN                     NaN
3              CAN  1959          0    NaN                     NaN
4  Channel Islands  1959          0    NaN                     NaN


We merge our internet access data. 

In [37]:
merged_df = pd.merge(merged_df, 
                     internet_df, 
                     on=['Country Code', 'Year'], 
                     how='left')

print(merged_df.head())

KeyError: 'Year'

We merge our religion data. We make these factors dummy variables since they are categorical.

In [ ]:
merged_df = pd.merge(merged_df, 
                     religion_df, 
                     left_on=['Country Code', 'Year'], 
                     right_on=['code', 'Year'],
                     how='left')

religion_dummies = pd.get_dummies(merged_df['topReligion'])
religion_dummies = religion_dummies.astype(int)
merged_df = pd.concat([merged_df, religion_dummies], axis=1)
merged_df.drop('topReligion', axis=1, inplace=True)

print(merged_df.head())

We turn the country names of the political and wealth data into country codes to merge. 

In [ ]:
political_df['Country'] = political_df['Country'].map(country_map).fillna(political_df['Country'])
wealth_df['Countries'] = wealth_df['Countries'].map(country_map).fillna(wealth_df['Countries'])

print(wealth_df.head(1))
print(political_df.head(1))

We merge our politcal data. 

In [ ]:
merged_df = pd.merge(merged_df, 
                     political_df, 
                     left_on=['Country Code', 'Year'],
                     right_on=['Country', 'Year'], how='left')
merged_df.drop(columns='Country', inplace=True)

print(merged_df.head())

We merge our wealth data. 

In [ ]:
merged_df = pd.merge(merged_df, 
                     wealth_df, 
                     left_on=['Country Code', 'Year'], 
                     right_on=['Countries', 'Year'], 
                     how='left')
merged_df.drop(columns='Countries', inplace=True)

print(merged_df.head())

Due to the splicing of other dataframes, we make everything_df our final dataframe as a copy of merge to create its own data.

In [ ]:
everything_df = merged_df.copy()
everything_df.head()

We check the types of the columns after modification.

In [ ]:
print(everything_df.iloc[:,0].unique())
everything_df.columns = [np.str_(col) for col in everything_df.columns]

for col in everything_df.columns:
    print(type(col))

We replace NaN values with the mean of the associated column

Imputer = SimpleImputer(strategy='mean')
columns_to_impute = ['Year', 'Average Years of Schooling', 'internet access',
                     'christianity_protestant', 'christianity_romancatholic',
                     'Democracy_Index', 'Wealth_share_percent']
everything_df[columns_to_impute] = imputer.fit_transform(everything_df[columns_to_impute])

We display the final dataframe.

In [ ]:
print(everything_df.head())

We save the dataframe as a csv file. 

In [ ]:
everything_df.to_csv("final_df", sep=',')

---

## Other Sources

1. Used for formatting of Markdown outside of what was covered in class: https://www.ibm.com/docs/en/db2-event-store/2.0.0?topic=notebooks-markdown-jupyter-cheatsheet 
1. Used while creating top_df to solve a SettingWithCopyError: https://github.com/pandas-dev/pandas/issues/39418 
1. Used to get country codes and corresponding countruy names: https://www.iban.com/country-codes